In [15]:
import etl_project_gdp as etl
import pandas as pd

In [16]:
list_of_gdp = etl.main()

[Extract] Found the target table. Parsing data...
[Extract] Extracted 209 rows.
[Extract] Saving data to data/Countries_by_GDP.json...
[Extract] Data successfully saved to data/Countries_by_GDP.json.
[ETL] Process completed successfully.
[Transform] Parsing HTML data...


In [17]:
list_of_gdp[list_of_gdp['gdp'] >= 100]

,country,type,gdp,year
0,United States,IMF,30337.16,2025
1,China,IMF,19534.89,2025
2,Germany,IMF,4921.56,2025
3,Japan,IMF,4389.33,2025
4,India,IMF,4271.92,2025
...,...,...,...,...
67,Uzbekistan,IMF,112.65,2024
68,Guatemala,IMF,112.37,2024
69,Oman,IMF,109.99,2024
70,Bulgaria,IMF,108.42,2024


In [33]:
regions = pd.read_json('data/cultural-geo-mapper.json')
merged = list_of_gdp.merge(regions, on='country', how='left')

def top_n_mean(group, n=5):
    # 그룹 내 GDP 상위 n개 선택
    top_n = group.nlargest(n, 'gdp')
    return top_n['gdp'].mean()

merged.groupby('region').apply(top_n_mean, n=5)

/var/folders/81/7j06ccbd657bxg70hxvk8d180000gn/T/ipykernel_53814/3107584315.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.groupby('region').apply(top_n_mean, n=5)


region
Central Asia           106.386
East Asia             5421.570
Eastern Europe        1066.600
Latin America          791.566
Middle East            598.134
North Africa           182.072
North America         8622.340
Oceania                436.658
South Asia            1043.380
Southeast Asia         711.890
Sub-Saharan Africa     197.530
Western Europe        3244.486
dtype: float64